In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import sklearn
from sklearn.preprocessing import MinMaxScaler
import unidecode
import os
import warnings
warnings.filterwarnings("ignore")

In [13]:
# Cargar datasets preprocesados
df_completo = pd.read_parquet(r'..\Datasets\dataset_completo.parquet')
df_actor = pd.read_parquet(r'..\Datasets\actor_dataset.parquet')
df_director = pd.read_parquet(r'..\Datasets\director_dataset.parquet')

In [14]:
# Función para normalizar el título antes de buscarlo en el dataset (incluso si hay diferencias en mayúsculas o acentos)
def limpiar_texto_completo(texto):
    if pd.isnull(texto):
        return None
    try:
        texto = str(texto).lower().strip()
        texto = ' '.join(texto.split())  # Eliminar espacios extra
        texto = unidecode.unidecode(texto)  # Eliminar caracteres especiales
    except Exception as e:
        print(f"Error procesando el texto: {texto}, Error: {e}")
        return None 
    return texto

# Aplicar limpieza de texto en nombres de actores y directores
df_actor['actor_name'] = df_actor['actor_name'].apply(limpiar_texto_completo)
df_director['director_name'] = df_director['director_name'].apply(limpiar_texto_completo)

# Crear la matriz de características basada en TF-IDF o características numéricas
# Suponiendo que ya tienes el TF-IDF entrenado
features_matrix = ...  # Esto debe generarse a partir de tu TF-IDF o características numéricas normalizadas

In [15]:
# Función de recomendación
def recomendacion(titulo, n_recomendaciones=5):
    try:
        # Limpiar y normalizar el título de la película
        titulo_normalizado = limpiar_texto_completo(titulo)
        
        # Verificar si el título existe en el dataset
        pelicula_idx = df_completo[df_completo['title'].str.contains(titulo_normalizado, case=False, na=False)].index
        
        if pelicula_idx.empty:
            return {"mensaje": f"El título '{titulo}' no se encuentra en el dataset."}
        
        # Obtener el índice de la película en el dataset
        idx = pelicula_idx[0]
        
        # Calcular las similitudes utilizando la matriz de características
        cosine_similarities = cosine_similarity([features_matrix[idx]], features_matrix).flatten()
        
        # Obtener los índices de las películas más similares y evitar repetir la misma película
        similar_indices = cosine_similarities.argsort()[::-1][1:n_recomendaciones + 1]
        
        # Obtener los títulos de las películas recomendadas
        recomendaciones = df_completo['title'].iloc[similar_indices].tolist()
        
        return {"recomendaciones": recomendaciones}
    
    except Exception as e:
        return {"mensaje": f"Error interno en la recomendación: {str(e)}"}

In [16]:
# Ruta relativa para la carpeta Datasets
datasets_dir = os.path.abspath(os.path.join('..', 'Datasets'))

# Verifica si el archivo Parquet existe en la ruta especificada
file_path = os.path.join(datasets_dir, 'features_matrix.parquet')

try:
    if os.path.exists(file_path):
        # Si el archivo existe, cargar la matriz desde el archivo Parquet
        features_df = pd.read_parquet(file_path)
        print(f"Archivo cargado desde: {file_path}")
    else:
        raise FileNotFoundError
except FileNotFoundError:
    # Si el archivo no existe o hay un error, genera y guarda la matriz de características
    print(f"Archivo no encontrado. Generando nueva matriz y guardando en: {file_path}")
    
    # Selecciona las columnas relevantes para la recomendación
    features = df_completo[['vote_average', 'vote_count']]

    # Normaliza las columnas seleccionadas
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)

    # Convertir la matriz de características a un DataFrame de pandas
    features_df = pd.DataFrame(features_scaled, columns=['vote_average', 'vote_count'])

    # Asegura que la carpeta Datasets exista antes de guardar
    os.makedirs(datasets_dir, exist_ok=True)

    # Guardar la matriz de características en un archivo Parquet en la ruta relativa
    features_df.to_parquet(file_path)
    print(f"Archivo guardado en: {file_path}")

except Exception as e:
    # Manejo de otros posibles errores
    print(f"Ocurrió un error: {e}")

Archivo cargado desde: c:\Users\Usuario\OneDrive\Escritorio\proyecto\Datasets\features_matrix.parquet


In [17]:
# Guardar el dataset en formato Parquet
df_completo.to_parquet(r'..\Datasets\dataset_completo.parquet')